In [32]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import ticker, cm
from math import sqrt, pi
from numpy import exp, cos, sin
from numpy.linalg import norm
import time

#Algorithm 3 BFGS Algorithm


Require: Starting point $x_0$, Stopping tolerance τ

1: Initialize k = 0, $B_0$ =??           ▷ Initialize Hessian approximation

2: while $∥∇f(x_k)∥_2 > τ$ do

3: Compute descent direction: $p_k = −B_k∇f(x_k)$

4: Choose step size: $α_k = arg min_{α≥0} f(x_k + αp_k)$

5: Update iterate: $x_{k+1} = x_k + α_kp_k$

6: Compute new gradient: $s_k = x_{k+1} − x_k, y_k = ∇f(x_{k+1}) − ∇f(x_k)$

7: Update Hessian approximation: B_{k+1} =?? ▷ Replace ?? by the update mentioned
in BFGS Theory

8: k = k + 1

9: Output: $x_k$


**$f(x) = f(x_1, x_2, x_3, ...., x_n) = \sum_i^{n-1} [4(x_i^2 − x_i+1)^2 + (x_i − 1)^2]$**

**$g(x) = g(x_1, x_2, x_3, ...., x_n) = \sum_i^n[(x_1 − x_i^2)^2 + (x_i − 1)^2]$**

**1. What is the minimizer and minimum function value of f(x) and g(x) ? Are both the function convex ? What
is a suitable initial choice of B (denoted by B0, i.e. Replacement of first ?? in the Algorithm 3)? Justify
with proper reasons.**

To find the minimizer and minimum function value of $ f(x) $ and $ g(x) $, we need to solve for the critical points of these functions and then check whether they correspond to minima.

### Minimizer and Minimum Function Value:

#### Function $ f(x) $:
The minimizer and minimum function value of $ f(x) $ can be found by setting its gradient to zero and solving for $ x $. However, due to the complexity of the function and the presence of multiple variables, finding an analytical solution may be challenging. Numerical optimization techniques like gradient descent or Newton's method can be used to find the minimizer and minimum function value.

#### Function $ g(x) $:
Similarly, for $( g(x) $), we need to find the critical points by setting its gradient to zero and solving for $( x $). Again, numerical optimization techniques may be needed due to the complexity of the function.

### Convexity:

#### Function $ f(x) $:
To determine if  f(x) is convex, we need to check if its Hessian matrix is positive semidefinite for all  x . Since the Hessian involves second-order derivatives of $ f(x) $, its computation and analysis can be complex and may require numerical methods or specialized software.

#### Function $ g(x) $:
Similar to $ f(x) $, we need to examine the Hessian of $ g(x) $ to determine its convexity.

### Initial Choice of $ B_0$  in Algorithm 3:

The initial choice of $B_0 $ in the BFGS algorithm is crucial for the convergence and efficiency of the algorithm. A suitable initial choice for $ B_0 $ is often the identity matrix or a positive definite matrix that is a good approximation of the Hessian matrix at the starting point. This choice helps ensure that the update formula in the BFGS algorithm generates a positive definite approximation of the Hessian at each iteration, maintaining the positive definiteness of the approximation throughout the optimization process.

In summary, finding the minimizer and minimum function value of $ f(x) $ and $( g(x) $ involves solving for critical points, which can be complex due to the functions' multivariate nature. Determining the convexity of the functions requires analyzing their Hessian matrices. A suitable initial choice for $ B_0 $ in the BFGS algorithm is typically the identity matrix or a positive definite approximation of the Hessian at the starting point to ensure convergence and stability.

**2. Implement Algorithm 3 for solving $min_{x∈R^n} f(x)$, Use backtracking line search with $α_0 = 0.9, ρ = 0.5$, γ = 0.5.
Take the starting point to be $x_0 = (0, 0, ..., 0)$. Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the BFGS method with backtracking line search.
Tabulate the time taken by BFGS method for each n.**

In [27]:
def fx(xk):
  length = len(xk)
  sum = 0
  for i in range(length-1):
    sum+= 4*(xk[i]**2 - xk[i+1])**2 + (xk[i]-1)**2
  return sum

def gradient_fx(xk):
  n = len(xk)
  grad = []
  grad.append( 16*xk[0]*(xk[0]**2 - xk[1]) + 2*(xk[0]-1) )
  for i in range(1, n-1):
    grad.append(  -8*(xk[i-1]**2 - xk[i]) + 16*xk[i]*(xk[i]**2 - xk[i+1]) + 2*(xk[i]-1) )
  grad.append(-8*(xk[n-2]**2 - xk[n-1]))
  return np.array(grad)

In [20]:
def get_alpha_bfgs(xk, alpha0, rho, gamma, Bk):
  alpha = alpha0
  pk = -gradient_fx(xk)
  while fx(xk + alpha*Bk@pk) > (fx(xk) + gamma*alpha*gradient_fx(xk)@Bk@pk):
    alpha = rho*alpha
  return alpha


def bfgs(x0, tau, alpha0, rho, gamma, max_iter=500):
  start_time = time.time()
  xk = np.copy(x0)
  n = len(x0)
  Bk = np.eye(n)
  count = 0
  pk = gradient_fx(xk)
  xks = []
  xks.append(xk)
  while (norm(pk)>tau):
    if count > max_iter:
      break

    alpha = get_alpha_bfgs(xk, alpha0, rho, gamma, Bk)
    xnext = xk - alpha*(Bk@pk)

    # print("new xk: ", xk[0:5])
    ## new Bk+1 computation
    sk = xnext - xk
    yk = gradient_fx(xnext) - gradient_fx(xk)
    # BFGS update formula
    Bk = np.dot((np.eye(len(xk)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(Bk, (np.eye(len(xk)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)

    xk = xnext
    pk = gradient_fx(xk)
    # print("grad is: ", pk[0:5])
    # print("grad norm is: ", norm(pk))
    xks.append(xk)
    count += 1

  end_time = time.time()
  time_elapsed = end_time - start_time
  return count, xk, fx(xk), xks, time_elapsed

In [21]:
alpha0 = 0.9
rho = 0.5
gamma = 0.5
ns = [1000, 2500, 5000, 7500, 10000]
tau = 2
for n in ns:
  x0 = [0 for i in range(n)]
  count, minimizer, minimum, xks, time_elapsed=bfgs(x0,tau, alpha0, rho, gamma)
  print(f"for n = {n}")
  print(f"total iterations: {count} | minimizer: {minimizer[0:5]} | minimum: {minimum} | time taken: {time_elapsed}")
  print("----------------------------------------------------------------")

for n = 1000
total iterations: 17 | minimizer: [0.99772206 0.99874487 0.99972807 0.99895936 1.00007376] | minimum: 0.1514899862210296 | time taken: 4.983297109603882
----------------------------------------------------------------
for n = 2500
total iterations: 19 | minimizer: [0.99840685 0.99967232 1.00024808 0.99757509 1.00077045] | minimum: 0.1276955357858959 | time taken: 46.2685272693634
----------------------------------------------------------------
for n = 5000
total iterations: 36 | minimizer: [0.99761426 0.99548031 0.990543   0.9904915  0.99011987] | minimum: 0.1585618341870785 | time taken: 590.1595966815948
----------------------------------------------------------------
for n = 7500
total iterations: 35 | minimizer: [1.00581333 1.00531036 1.00092941 1.00088032 1.00268388] | minimum: 0.19255899400483575 | time taken: 1834.519108057022
----------------------------------------------------------------
for n = 10000
total iterations: 32 | minimizer: [0.99828304 0.99509938 0.995

**3. Implement Algorithm 3 for solving $min_{x∈R^n} g(x)$, Use backtracking line search with α0 = 0.9, ρ = 0.5, γ = 0.5.
Take the starting point to be x0 = (0, 0, ..., 0). Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the BFGS method with backtracking line search.
Tabulate the time taken by BFGS method for each n.**


**$f(x) = f(x_1, x_2, x_3, ...., x_n) = \sum_i^{n-1} [4(x_i^2 − x_i+1)^2 + (x_i − 1)^2]$**

**$g(x) = g(x_1, x_2, x_3, ...., x_n) = \sum_i^n[(x_1 − x_i^2)^2 + (x_i − 1)^2]$**

In [36]:
def gx(xk):
  length = len(xk)
  sum = 0
  for i in range(length-1):
    sum+= (xk[0] - xk[i]**2)**2 + (xk[i]-1)**2
  return sum

def gradient_gx(xk):
  n = len(xk)
  grad = []
  grad.append((2*(xk[0] - xk[0]**2) )*(1-2*xk[0]) + 2*(xk[0]-1))

  for i in range(1, n):
    grad.append( -4*xk[i]*(xk[0] - xk[i]**2)+2*(xk[i]-1) )

  return np.array(grad)

In [37]:
def get_alpha_bfgs(xk, alpha0, rho, gamma, Bk):
  alpha = alpha0
  pk = -gradient_gx(xk)
  while gx(xk + alpha*Bk@pk) > (gx(xk) + gamma*alpha*gradient_gx(xk)@Bk@pk):
    alpha = rho*alpha
  return alpha


def bfgs(x0, tau, alpha0, rho, gamma, max_iter=500):
  start_time = time.time()
  xk = np.copy(x0)
  n = len(x0)
  Bk = np.eye(n)
  count = 0
  pk = gradient_gx(xk)
  xks = []
  xks.append(xk)
  while (norm(pk)>tau):
    if count > max_iter:
      break

    alpha = get_alpha_bfgs(xk, alpha0, rho, gamma, Bk)
    xnext = xk - alpha*(Bk@pk)


    sk = xnext - xk
    yk = gradient_gx(xnext) - gradient_gx(xk)

    Bk = np.dot((np.eye(len(xk)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(Bk, (np.eye(len(xk)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)

    xk = xnext
    pk = gradient_gx(xk)

    xks.append(xk)
    count += 1

  end_time = time.time()
  time_elapsed = end_time - start_time
  return count, xk, fx(xk), xks, time_elapsed

In [38]:
alpha0 = 0.9
rho = 0.5
gamma = 0.5
ns = [1000, 2500, 5000, 7500, 10000]
tau = 2
for n in ns:
  x0 = [0 for i in range(n)]
  count, minimizer, minimum, xks, time_elapsed = bfgs(x0,tau, alpha0, rho, gamma)
  print(f"for n = {n}")
  print(f"total iterations: {count} | minimizer: {minimizer[0:5]} | minimum: {minimum} | time taken: {time_elapsed}")

<ipython-input-37-ba0675ba863b>:29: RuntimeWarning: invalid value encountered in divide
  Bk = np.dot((np.eye(len(xk)) - np.outer(sk, yk) / np.dot(yk, sk)), np.dot(Bk, (np.eye(len(xk)) - np.outer(yk, sk) / np.dot(yk, sk)))) + np.outer(sk, sk) / np.dot(yk, sk)


for n = 1000
total iterations: 4 | minimizer: [nan nan nan nan nan] | minimum: nan | time taken: 1.1399421691894531
for n = 2500
total iterations: 4 | minimizer: [nan nan nan nan nan] | minimum: nan | time taken: 11.822689771652222
for n = 5000
total iterations: 4 | minimizer: [nan nan nan nan nan] | minimum: nan | time taken: 73.32806253433228
for n = 7500
total iterations: 4 | minimizer: [nan nan nan nan nan] | minimum: nan | time taken: 220.5196189880371
for n = 10000
total iterations: 4 | minimizer: [nan nan nan nan nan] | minimum: nan | time taken: 497.45763874053955


**3. Implement Algorithm 2 for solving minx∈Rn f(x), Use backtracking line search with α0 = 0.9, ρ = 0.5, γ = 0.5.
Take the starting point to be $x_0 = (0, 0, ..., 0)$. Take n ∈ {1000, 2500, 5000, 7500, 10000}, find minimizer of the
objective function in each case and compute the time taken by the Newton’s method with backtracking line
search. Tabulate the time taken by Newton’s method for each n.**

In [40]:
import numpy as np
import time

def f(x):
    n = len(x)
    return sum([4 * (x[i]**2 - x[i+1])**2 + (x[i] - 1)**2 for i in range(n-1)])

def gradient_f(x):
    n = len(x)
    grad = np.zeros(n)
    for i in range(n-1):
        grad[i] += 8 * (x[i]**2 - x[i+1]) * x[i] - 8 * (x[i]**2 - x[i+1])
        grad[i] += 2 * (x[i] - 1)
        grad[i+1] += -8 * (x[i]**2 - x[i+1])
    grad[-1] += 2 * (x[-1] - 1)
    return grad

def hessian_f(x):
    n = len(x)
    hess = np.zeros((n, n))
    for i in range(n-1):
        hess[i][i] += 8 * (3 * x[i]**2 - 2 * x[i] * x[i+1] - x[i+1])
        hess[i][i+1] += -8 * x[i]
        hess[i+1][i] += -8 * x[i]
        hess[i+1][i+1] += 8 * x[i]
    hess[-1][-1] = 2
    return hess

def backtracking_line_search(f, grad_f, x, delta_x, alpha=0.9, rho=0.5, gamma=0.5):
    eta = 1.0
    while f(x + eta * delta_x) > f(x) + alpha * eta * np.dot(grad_f(x), delta_x):
        eta *= rho
    return eta

def newtons_method_with_line_search(f, grad_f, hess_f, x0, tau, alpha=0.9, rho=0.5, gamma=0.5, epsilon=1e-6):
    x = x0.copy()
    start_time = time.time()
    while np.linalg.norm(grad_f(x)) > tau:
        hess = hess_f(x)
        try:
            inv_hess = np.linalg.inv(hess + epsilon * np.eye(len(x)))
        except np.linalg.LinAlgError:
            # If the Hessian is singular, we use a different approach
            # or modify the Hessian matrix to make it non-singular
            break
        delta_x = -inv_hess.dot(grad_f(x))
        eta = backtracking_line_search(f, grad_f, x, delta_x, alpha, rho, gamma)
        x += eta * delta_x
    end_time = time.time()
    return x, end_time - start_time


# Main loop
n_values = [1000, 2500, 5000, 7500, 10000]
minimizers = []
times = []

for n in n_values:
    x0 = np.zeros(n)
    tau = 1e-9
    minimizer, time_taken = newtons_method_with_line_search(f, gradient_f, hessian_f, x0, tau)
    minimizers.append(minimizer)
    times.append(time_taken)

# Tabulate the results
print("n\tMinimizer\tTime (s)")
for i, n in enumerate(n_values):
    print(f"{n}\t{minimizers[i]}\t{times[i]}")


KeyboardInterrupt: 